In [3]:
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import faiss

In [4]:
# === Step 1: Prepare Sample Data ===
documents = [
    "The Eiffel Tower is located in Paris, France.",
    "Python is a popular programming language for data science.",
    "The Great Wall of China is a historic fortification.",
    "Machine learning is a subset of artificial intelligence."
]
query = "Where is the Eiffel Tower?"

In [5]:
# === Step 2: Initialize Sentence Transformer for Retriever ===
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = retriever_model.encode(documents)

# Create FAISS index
embedding_dim = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(doc_embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Ramro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ramro\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# === Step 3: Retrieve Relevant Documents ===
query_embedding = retriever_model.encode([query])
k = 2  # Number of documents to retrieve
distances, indices = index.search(query_embedding, k)
retrieved_docs = [documents[idx] for idx in indices[0]]

print("Retrieved Documents:")
for doc in retrieved_docs:
    print(f"- {doc}")

Retrieved Documents:
- The Eiffel Tower is located in Paris, France.
- The Great Wall of China is a historic fortification.


In [8]:
# === Step 4: Initialize Generator ===
generator_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
generator_tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Prepare input for the generator
input_text = " ".join(retrieved_docs) + f" Query: {query}"
tokenized_input = generator_tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)

# Generate response
generated_output = generator_model.generate(tokenized_input, max_length=50, num_beams=2, early_stopping=True)
response = generator_tokenizer.decode(generated_output[0], skip_special_tokens=True)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

C:\Users\Ramro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ramro\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [9]:
print("\nGenerated Response:")
print(response)


Generated Response:
is located in Paris, France.
